In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
##########################
# Mathematical Functions #
##########################

def coordinate_conversion(manipulation_matrix: np.ndarray, alpha: float, x0: float, y0: float) -> np.ndarray:
    """Converts coordinates from the origin to the local coordinate system of the nanowire.
    
    Note: The alpha angle must be in radians. If it's in degrees, it should be converted to radians before passing into this function.
    
    Args:
        manipulation_matrix (np.ndarray): The original coordinates to convert.
        alpha (float): The angle of rotation in radians.
        x0 (float): The x-coordinate for the translation.
        y0 (float): The y-coordinate for the translation.

    Returns:
        transformed_manipulation_matrix (np.ndarray): The converted coordinates.
    """

    rotation_matrix = np.array([[np.cos(alpha), np.sin(alpha)], [-np.sin(alpha), np.cos(alpha)]])
    transformed_manipulation_matrix = np.empty_like(manipulation_matrix)
    new_manipulation_matrix = np.zeros((manipulation_matrix.shape[0], 2, 3))
    affine_matrix = np.vstack((rotation_matrix, [x0, y0]))

    # Add a third coordinate of 1 to each point in the manipulation matrix
    for n in range(len(manipulation_matrix)):
        new_manipulation_matrix[n] = np.hstack((manipulation_matrix[n], np.ones((manipulation_matrix[n].shape[0], 1))))
        transformed_manipulation_matrix[n] = new_manipulation_matrix[n] @ affine_matrix

    return transformed_manipulation_matrix #Should not return with check_direction() as it is should not always be from left to right

def check_direction(position_matrix: np.ndarray) -> np.ndarray:
    """Ensures the nanowire's direction goes from left to right.

    Args:
        position_matrix (np.ndarray): The original position matrix of the nanowire.

    Returns:
        position_matrix (np.ndarray): The position matrix with guaranteed left-to-right direction.
    """
    if position_matrix[1,0] - position_matrix[0,0] < 0:
            position_matrix = position_matrix[[1, 0]]

    return position_matrix

def ceil_to_nearest_even(n: float) -> int:
    """Rounds up to the nearest even number.

    Args:
        n (float): The original number.

    Returns:
        (int): The nearest even number that is equal or larger than n.
    """
    return int(np.ceil(n / 2.) * 2)

def calculate_angle_difference(alpha_1: float, alpha_2: float) -> float:
    """Calculates the difference between two angles.

    Args:
        alpha_1 (float): The first angle. [radians]
        alpha_2 (float): The second angle to compare with. [radians]

    Returns:
        angle_difference (float): The absolute difference between the two angles. [radians]
    """
    
    angle_difference = np.abs(alpha_1 - alpha_2)

    if angle_difference > np.pi:
        angle_difference -= np.pi

    if angle_difference > np.pi/2:
        angle_difference -= np.pi
    return angle_difference

def create_position_matrix(x1: float, y1: float, x2: float, y2: float) -> np.ndarray:
    """Create a position matrix for a given pair of points.
    
    Args:
    x1, y1 (float): The coordinates of the first point.
    x2, y2 (float): The coordinates of the second point.

    Returns:
    position_matrix (np.ndarray): A 2D array with the coordinates of the points.
    """
    return  check_direction(np.array([[x1, y1], [x2, y2]]))

def create_random_position_matrix(size: float) -> np.ndarray:
    """Creates a random position matrix for a nanowire within a given size of image. The lengths of the nanowires are random within that range.

    Args:
        size (float): The maximum size for the nanowire's coordinates.

    Returns:
        position_matrix (np.ndarray): The random position matrix for the nanowire.
    """
    head_position, tail_position = np.random.rand(2, 2) * size

    position_matrix = create_position_matrix(*head_position, *tail_position)

    return check_direction(position_matrix)

def find_wire_parameters(position_matrix: np.ndarray) -> tuple:
    """Find the parameters of a wire based on its position matrix.
     - alpha; the angle with respect to the x axis, clockwise
     - length; the length of the wire
     - center_position; the coordinates [x,y] of its center

    position_matrix = | initial_x     initial_y   |
                      | end_x         end_y       |

    Args:
    position_matrix (np.ndarray): A 2D array with the coordinates of the wire ends.

    Returns:
    alpha, length, center (tuple): The parameters of the wire: angle, length and center.
    """

    dx = position_matrix[1,0] - position_matrix[0,0]
    dy = position_matrix[1,1] - position_matrix[0,1]

    alpha = np.arctan(dy/dx)
    if np.isnan(alpha):
        alpha = 0

    length = np.sqrt(dx**2 + dy**2)

    center_position = [dx/2 + position_matrix[0,0] , dy/2 + position_matrix[0,1]]

    return alpha, length, center_position

In [ ]:
#####################
# Ploting functions #
#####################

def plot_vector(position_matrix: np.ndarray, color: str='black', title: str='Nanowire Position', ax=None, label=None) -> plt.Axes:
    """Plot a given vector in 2D.
    
    Args:
    position_matrix (np.ndarray): A 2D array representing the vector.
    color (str, optional): Color of the vector to be plotted.
    title (str, optional): Title of the plot.
    ax (plt.Axes, optional): An instance of Axes in matplotlib, on which the vector will be plotted. 

    Returns:
    ax (plt.Axes): The Axes object with the plot.
    """
    head_x = position_matrix[0,0]
    head_y = position_matrix[0,1]
    tail_x = position_matrix[1,0]
    tail_y = position_matrix[1,1]
    
    if ax is None:
        fig, ax = plt.subplots(figsize=(30, 10))

        ax.set_xlim(0, np.abs(head_x - tail_x) + 1)
        ax.set_ylim(np.abs(head_y - tail_y) + 1, 0)
        
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_title(title)
        
    ax.arrow(head_x, head_y, tail_x - head_x, tail_y - head_y, head_width=0.05, head_length=0.02, color=color, label=label)

    return ax

def plot_PPVs(matrix_PPVs: np.ndarray, ax: plt.Axes = None) -> plt.Axes:
    """Plots the Parallel Pushing Vectors (PPVs).

    Args:
        matrix_PPVs (np.ndarray): The matrix of PPVs to be plotted.
        ax (plt.Axes, optional): An existing matplotlib Axes object to draw on. Default is None, which creates a new plot.

    Returns:
        ax (plt.Axes): The matplotlib Axes object with the PPVs plotted.
    """

    if ax is None:
        # Create plot if no axis object is provided
        fig, ax = plt.subplots(figsize=(30,10))
        # Configure plot
        ax.set_aspect('equal', adjustable='box')
        ax.grid(True)
        ax.set_title('Initial and Transformed Vectors')
        ax.set_xlabel('x')
        ax.set_ylabel('y')

    for n in range(len(matrix_PPVs)):
        dx = matrix_PPVs[n,1,0] - matrix_PPVs[n,0,0]
        dy = matrix_PPVs[n,1,1] - matrix_PPVs[n,0,1]

        ax.arrow(matrix_PPVs[n,0,0], matrix_PPVs[n,0,1], dx, dy, head_width=0.05, head_length=0.02, fc='blue', ec='blue',length_includes_head=True)

    return ax

In [ ]:
#############################
# Transformations functions #
#############################

def create_single_translation_PPVs(initial_length: float, delta_x: float, delta_y: float, direction_y: int, wire_radius: float, probe_radius: float , recession_dist: float) -> np.ndarray:

    """Creates the PPVs for a single translation, centered at the origin.

    Args:
        initial_length (float): The length of the nanowire.
        delta_x (float): The interval between adjacent PPVs.
        delta_y (float): The effective length of each vector.
        direction_y (int): The direction along the y axis (1 or -1).
        wire_radius, probe_radius, recession_dist (float): Parameters used in the translation.

    Returns:
        parallel_pushing_vectors (np.ndarray): The PPVs for a single translation.
    """

    num_PPVs = ceil_to_nearest_even(initial_length/delta_x)
    delta_x = initial_length/num_PPVs

    # Initialize the arrays
    starting_points = np.zeros((2, num_PPVs))
    ending_points = np.zeros((2, num_PPVs))
    parallel_pushing_vectors = np.zeros((num_PPVs, 2, 2))

    for n in range(num_PPVs):
        starting_points[0,n] = 0.5*initial_length - n * delta_x - delta_x/2
        starting_points[1,n] = -(wire_radius + probe_radius + recession_dist) * direction_y

        ending_points[0,n] = starting_points[0,n]
        ending_points[1,n] = starting_points[1,n] + (recession_dist + delta_y) * direction_y

        parallel_pushing_vectors[n] = np.array([[starting_points[0,n], starting_points[1,n]], [ending_points[0,n], ending_points[1,n]]])

    return parallel_pushing_vectors

def translate_nanowire(initial_position_matrix: np.ndarray, final_center_position: list, delta_x: float, delta_y: float, wire_radius: float, probe_radius: float, recession_dist: float) -> np.ndarray:
    """Translates a nanowire from an initial position to a final position.

    Args:
        initial_position_matrix (np.ndarray): The initial position of the nanowire.
        final_center_position (list, size 2): The final center position of the nanowire.
        delta_x (float): The interval in the x-direction.
        delta_y (float): The interval in the y-direction.
        wire_radius, probe_radius, recession_dist (float): Parameters used in the translation.

    Returns:
        manipulation_matrix (np.ndarray): The matrix containing the PPVs for the translation of the nanowire.
    """

    initial_alpha, initial_length, initial_center_position = find_wire_parameters(initial_position_matrix)

    direction_x = np.sign(final_center_position[0] - initial_center_position[0])
    direction_y = np.sign(final_center_position[1] - initial_center_position[1])
    
    center_to_center_position_matrix = create_position_matrix(*initial_center_position, *final_center_position)
    center_to_center_alpha, distance_to_travel, _ = find_wire_parameters(center_to_center_position_matrix)

    # Here we do delta_y - wire_radius - probe_radius because that is the effective translation we operate on the nanowire.
    # See Sn[Y] + recession_dist + delta_y and replace by the expression of Sn[Y]

    number_translations = int(np.floor(distance_to_travel/(delta_y-wire_radius-probe_radius)))
    num_PPVs = ceil_to_nearest_even(initial_length/delta_x)
    delta_x = initial_length/num_PPVs
 
    manipulation_matrix = np.zeros((number_translations + 1, num_PPVs, 2, 2))
    translation_vectors = create_single_translation_PPVs(initial_length, delta_x, delta_y, direction_y, wire_radius, probe_radius, recession_dist)
    
    new_position_matrix = initial_position_matrix
    new_center = initial_center_position

    for i in range(number_translations):
        manipulation_matrix[i] = coordinate_conversion(translation_vectors, initial_alpha, *new_center)

        displacement_vector = np.array([np.cos(center_to_center_alpha), np.sin(center_to_center_alpha)]) * (delta_y - wire_radius - probe_radius) * direction_x
        new_position_matrix = coordinate_conversion(new_position_matrix[None, :] - initial_center_position + displacement_vector, 0, *initial_center_position)[0]
        new_center = find_wire_parameters(new_position_matrix)[2]

    center_difference_position_matrix = create_position_matrix(*new_center, *final_center_position)
    final_distance = find_wire_parameters(center_difference_position_matrix)[1]
    final_translation_vectors = create_single_translation_PPVs(initial_length, delta_x, final_distance, direction_y, wire_radius, probe_radius, recession_dist)
    
    final_displacement_vector = np.array([np.cos(center_to_center_alpha), np.sin(center_to_center_alpha)]) * (final_distance - wire_radius - probe_radius) * direction_x
    new_position_matrix = coordinate_conversion(new_position_matrix[None, :] - initial_center_position + final_displacement_vector, 0, *initial_center_position)[0]
    manipulation_matrix[-1] = coordinate_conversion(final_translation_vectors, initial_alpha, *new_center)

    return manipulation_matrix

def create_single_rotation_PPVs(initial_length: float, direction_rotation: int, delta_x: float, delta_theta: float, wire_radius: float, probe_radius: float, recession_dist: float) -> np.ndarray:
    """Creates the PPVs for a single rotation, centered at the origin.

    Args:
        initial_length (float): The length of the nanowire.
        direction_rotation (int): The direction of rotation (1 or -1).
        delta_x (float): The interval between adjacent PPVs.
        delta_theta (float): The interval of rotation.
        wire_radius, probe_radius, recession_dist (float): Parameters used in the translation.

    Returns:
        parallel_pushing_vectors (np.ndarray): The PPVs for a single rotation.
    """

    num_PPVs = ceil_to_nearest_even(initial_length/delta_x)
    starting_points = np.zeros((2, num_PPVs))
    ending_points = np.zeros((2, num_PPVs))
    parallel_pushing_vectors = np.zeros((num_PPVs, 2, 2))

    for n in range(num_PPVs):
        if 0 <= n < num_PPVs/2:
            starting_points[0,n] = initial_length - n * delta_x + (wire_radius + probe_radius + recession_dist) * np.tan(delta_theta/2)
            starting_points[1,n] = -(wire_radius + probe_radius + recession_dist) * direction_rotation

            ending_points[0,n] = 0.5 * initial_length + (0.5 * initial_length - n * delta_x) * np.cos(delta_theta) + (wire_radius + probe_radius) * np.sin(delta_theta/2) * direction_rotation
            ending_points[1,n] = (0.5 * initial_length - n * delta_x) * np.sin(delta_theta) - (wire_radius + probe_radius) * np.cos(delta_theta/2) * direction_rotation

        elif num_PPVs/2 <= n <= num_PPVs:
            starting_points[0,n] = initial_length - starting_points[0,n-int(num_PPVs/2)]
            starting_points[1,n] = (wire_radius + probe_radius + recession_dist) * direction_rotation

            ending_points[0,n] = initial_length - ending_points[0,n-int(num_PPVs/2)]
            ending_points[1,n] = - ending_points[1,n-int(num_PPVs/2)]

        parallel_pushing_vectors[n] = np.array([[starting_points[0,n], starting_points[1,n]], [ending_points[0,n], ending_points[1,n]]])
    return parallel_pushing_vectors

def rotate_nanowire(initial_position_matrix: np.ndarray, final_alpha: float, delta_x: float, delta_theta: float, wire_radius: float, probe_radius: float, recession_dist: float, do_plot: bool=False, ax=None) -> np.ndarray:
    """Rotate a nanowire from its initial position to a final angle.
    
    Args:
        initial_position_matrix (np.ndarray): A 2D array with the coordinates of the wire ends.
        final_alpha (float): The final angle to be reached. Taken with respect to the x axis and clockwise.
        delta_x (float): The interval in the x-direction.
        delta_theta (float): The interval in the direction of rotation.
        wire_radius, probe_radius, recession_dist (float): Parameters used in the rotation.
        do_plot (bool): A flag to indicate if the plot should be shown.
        ax (plt.Axes, optional): An instance of Axes in matplotlib, on which the vector will be plotted.

    Returns:
        manipulation_matrix (np.ndarray): The matrix containing the PPVs for the rotation of the nanowire.
    """
    
    angle_difference = 0

    initial_alpha, initial_length, initial_center_position = find_wire_parameters(initial_position_matrix)
    direction_rotation = np.sign(final_alpha - initial_alpha)

    relative_angle = np.abs(final_alpha - initial_alpha)

    if relative_angle > np.pi/2:
        relative_angle = np.pi - relative_angle
        direction_rotation = -direction_rotation

    number_rotations = int(np.floor(np.abs(relative_angle / delta_theta)))
    num_PPVs = ceil_to_nearest_even(initial_length/delta_x)

    manipulation_matrix = np.zeros((number_rotations + 1, num_PPVs, 2, 2))
    rotation_vectors = create_single_rotation_PPVs(initial_length, direction_rotation, delta_x, delta_theta*direction_rotation, wire_radius, probe_radius, recession_dist)

    new_alpha = initial_alpha
    if new_alpha > np.pi:
        new_alpha -= np.pi

    new_position_matrix = initial_position_matrix
    new_x0, new_y0 = new_position_matrix[0,:]

    for i in range(number_rotations):
        manipulation_matrix[i] = coordinate_conversion(rotation_vectors, new_alpha, new_x0, new_y0)
        new_position_matrix = coordinate_conversion(np.array([new_position_matrix]) - initial_center_position, delta_theta*direction_rotation, initial_center_position[0], initial_center_position[1])[0]
        new_position_matrix = check_direction(new_position_matrix)

        new_alpha = find_wire_parameters(new_position_matrix)[0]
        new_x0, new_y0 = new_position_matrix[0,:]
        angle_difference = calculate_angle_difference(final_alpha, new_alpha)

        if do_plot:
            plot_vector(new_position_matrix, color='green', ax=ax)

    # Make a final rotation with what is remaining to achieve the exact desired angle
    final_rotation_vectors = create_single_rotation_PPVs(initial_length, direction_rotation, delta_x, angle_difference*direction_rotation, wire_radius, probe_radius, recession_dist)
    new_position_matrix = coordinate_conversion(np.array([new_position_matrix]) - initial_center_position, angle_difference*direction_rotation, initial_center_position[0], initial_center_position[1])[0]
    manipulation_matrix[-1] = coordinate_conversion(final_rotation_vectors, new_alpha, new_x0, new_y0)
    if do_plot:
        plot_vector(new_position_matrix, color='green', ax=ax)

    return manipulation_matrix

def move_nanowire(initial_position_matrix: np.ndarray, 
                  final_center_position: list, 
                  final_alpha: float, 
                  delta_x: float, 
                  delta_y: float, 
                  delta_theta: float, 
                  wire_radius: float, 
                  probe_radius: float, 
                  recession_dist: float, 
                  ax=None) -> np.ndarray:
    """Move a nanowire from its initial position to a final position and angle in the following 3 steps:
    
    Steps:
        1. Rotation to align the direction of translation with the center of the final position
        2. Translation to match the center of the nanowire with the desired final position
        3. Final rotation to match the desired final angle
    
    Args:
        initial_position (np.ndarray): A 2D array with the coordinates of the wire ends.
        final_center (list of float, size 2): A list of size 2 with the final coordinates of the center of the wire.
        final_angle (float): The final angle to be reached.
        delta_x, delta_y, delta_theta, wire_radius, probe_radius, recession_dist (float): Parameters used in the movement.
        ax (plt.Axes, optional): An instance of Axes in matplotlib, on which the vector will be plotted.


    Returns:
        manipulation_matrix (np.ndarray): The matrix containing the PPVs for the complete movement of the nanowire.
    """

    initial_alpha, _ , initial_center_position = find_wire_parameters(initial_position_matrix)

    center_to_center_position_matrix = create_position_matrix(*initial_center_position, *final_center_position)
    center_to_center_position_matrix = check_direction(center_to_center_position_matrix)
    center_to_center_alpha = find_wire_parameters(center_to_center_position_matrix)[0]

    direction_rotation = np.sign(center_to_center_alpha - initial_alpha)

    #1. Rotation
    manipulation_matrix = rotate_nanowire(initial_position_matrix, center_to_center_alpha - np.pi/2 * direction_rotation, delta_x, delta_theta, wire_radius, probe_radius, recession_dist, do_plot=True, ax=ax)

    # Rotate the coordinates to follow the manipulation
    relative_angle = center_to_center_alpha - np.pi/2 - initial_alpha
    rotated_position_matrix = coordinate_conversion(np.array([initial_position_matrix]) - initial_center_position, relative_angle, initial_center_position[0], initial_center_position[1])[0]
    rotated_position_matrix = check_direction(rotated_position_matrix)

    #2. Translation
    translation_matrix = translate_nanowire(rotated_position_matrix, final_center_position, delta_x, delta_y, wire_radius, probe_radius, recession_dist)
    manipulation_matrix = np.concatenate((manipulation_matrix, translation_matrix), axis=0)

    # Translate the coordinates to follow the manipulation
    translated_position_matrix = coordinate_conversion(np.array([rotated_position_matrix]), 0, final_center_position[0] - initial_center_position[0], final_center_position[1] - initial_center_position[1])[0]
    translated_position_matrix = check_direction(translated_position_matrix)

    #3. Final rotation
    final_rotation_matrix = rotate_nanowire(translated_position_matrix, final_alpha, delta_x, delta_theta, wire_radius, probe_radius, recession_dist, do_plot=True, ax=ax)
    manipulation_matrix = np.concatenate((manipulation_matrix, final_rotation_matrix), axis=0)

    return manipulation_matrix

def ask_for_coordinates(optional_prompt: str = "") -> np.ndarray:
    """Asks the user to input four floats for the head and tail coordinates of a nanowire.

    Args:
        optional_prompt (str, optional): An optional prompt message to display. Default is an empty string.

    Returns:
        position_matrix (np.ndarray): The position matrix created from the user's inputs.
    """
    position_parts = []

    # Keep asking until we get four floats
    while len(position_parts) != 4:
        position_input = input(f'{optional_prompt} Enter head_x, head_y, tail_x, tail_y separated by commas: ')
        position_parts = position_input.split(',')

        # Check if we got exactly four parts
        if len(position_parts) != 4:
            print('Error: You must enter exactly four numbers separated by commas.')
            continue

        # Check if each part can be converted to a float
        for part in position_parts:
            try:
                float(part)
            except ValueError:
                print('Error: each part must be a float')
                position_parts = []
                break

    # At this point, we know that we have exactly four floats
    head_position_x = float(position_parts[0])
    head_position_y = float(position_parts[1])
    tail_position_x = float(position_parts[2])
    tail_position_y = float(position_parts[3])

    position_matrix = create_position_matrix(head_position_x, head_position_y, tail_position_x, tail_position_y)
    return check_direction(position_matrix)

In [ ]:
##############################
# Test for translations only #
##############################
try:
    index += 1
except NameError:
    index = 0

np.random.seed(index)
print(f'{index=}')

initial_test_position_matrix = create_random_position_matrix(2)
final_test_position_matrix = create_random_position_matrix(2)

initial_alpha, initial_length, initial_center_position = find_wire_parameters(initial_test_position_matrix)
final_alpha, final_length, final_center_position = find_wire_parameters(final_test_position_matrix)

center_to_center_position_matrix = create_position_matrix(initial_center_position[0], initial_center_position[1], final_center_position[0], final_center_position[1])
center_to_center_alpha = find_wire_parameters(center_to_center_position_matrix)[0]

relative_angle = center_to_center_alpha - np.pi/2 - initial_alpha

rotated_test_position_matrix = coordinate_conversion(np.array([initial_test_position_matrix]) - initial_center_position, relative_angle, initial_center_position[0], initial_center_position[1])[0]
rotated_test_position_matrix = check_direction(rotated_test_position_matrix)

rotated_alpha, rotated_length, rotated_center_position = find_wire_parameters(rotated_test_position_matrix)

fig, ax = plt.subplots(figsize=(10, 10))

image_size_x = 2 # Update to scan size
image_size_y = 2 # Update to scan size

ax.set_xlim(0, image_size_x)  
ax.set_ylim(image_size_y, 0)  
ax.set_aspect('equal', adjustable='box')
ax.grid(True)
ax.set_title('Initial and Transformed Vectors')
ax.set_xlabel('x')
ax.set_ylabel('y')

delta_x = 0.1
delta_y = 0.1
delta_theta = 0.35
wire_radius = 0.02
probe_radius = 0.012 
recession_dist = 0.1 

test_manipulation_matrix = translate_nanowire(rotated_test_position_matrix, final_center_position, delta_x, delta_y, wire_radius, probe_radius, recession_dist)
translated_position_matrix = coordinate_conversion(np.array([rotated_test_position_matrix]), 0, final_center_position[0] - initial_center_position[0], final_center_position[1] - initial_center_position[1])[0]
translated_position_matrix = check_direction(translated_position_matrix)

for i in range(len(test_manipulation_matrix)):
    plot_PPVs(test_manipulation_matrix[i], ax=ax)
ax.get_children()[-11].set_label("Pushing vectors")

plot_vector(initial_test_position_matrix, ax=ax, label="Initial vector")
plot_vector(rotated_test_position_matrix, color='green', ax=ax, label="Rotated vector")
plot_vector(center_to_center_position_matrix, color='orange', ax=ax, label="Center to center vector")
plot_vector(translated_position_matrix, color='c', ax=ax, label="Translated vector")
plot_vector(final_test_position_matrix, color='red', ax=ax, label="Final vector")

ax.legend()

plt.savefig('Translation.svg')  

with open('output.txt', 'w') as f:
    for operation_number in range(len(test_manipulation_matrix)):
        for polyline in test_manipulation_matrix[operation_number]:
            f.write('Polyline:\n')
            f.write(','.join(['[{:.8f}, {:.8f}]'.format(polyline[i, 0], polyline[i, 1]) for i in range(2)]))
            f.write('\n\n')

In [ ]:
###########################
# Test for rotations only #
###########################

try:
    index += 1
except NameError:
    index = 0

np.random.seed(index)
print(f'{index=}')

initial_test_position_matrix = create_random_position_matrix(2)
final_test_position_matrix = create_random_position_matrix(2)

initial_alpha = find_wire_parameters(initial_test_position_matrix)[0]
final_alpha = find_wire_parameters(final_test_position_matrix)[0]


fig, ax = plt.subplots(figsize=(10, 10))

image_size_x = 2.5 # Update to scan size
image_size_y = 2.5 # Update to scan size

ax.set_xlim(0, image_size_x)
ax.set_ylim(image_size_y, 0)  
ax.set_aspect('equal', adjustable='box')
ax.grid(True)
ax.set_title('Initial and Transformed Vectors')
ax.set_xlabel('x')
ax.set_ylabel('y')

delta_x = 0.2
delta_y = 0.1
delta_theta = 0.1
wire_radius = 0.02
probe_radius = 0.012 
recession_dist = 0.1 

test_manipulation_matrix = rotate_nanowire(initial_test_position_matrix, final_alpha, delta_x, delta_theta, wire_radius, probe_radius, recession_dist, do_plot=True, ax=ax)

for i in range(len(test_manipulation_matrix)):
    ax = plot_PPVs(test_manipulation_matrix[i], ax=ax)

ax.get_children()[0].set_label("Intermediate vectors")
ax.get_children()[-11].set_label("Pushing vectors")


plot_vector(initial_test_position_matrix, ax=ax, label="Initial vector")
plot_vector(final_test_position_matrix, ax=ax, color='red', label="Final vector")




ax.legend()

plt.savefig('Rotation.svg')  


with open('output.txt', 'w') as f:
    for operation_number in range(len(test_manipulation_matrix)):
        for polyline in test_manipulation_matrix[operation_number]:
            f.write('Polyline:\n')
            f.write(','.join(['[{:.8f}, {:.8f}]'.format(polyline[i, 0], polyline[i, 1]) for i in range(2)]))
            f.write('\n\n')

In [ ]:
##########################
# Test for complete move #
##########################

try:
    index += 1
except NameError:
    index = 0

np.random.seed(index)
print(f'{index=}')

initial_test_position_matrix = create_random_position_matrix(2)
final_test_position_matrix = create_random_position_matrix(2)

initial_alpha, initial_length, initial_center_position = find_wire_parameters(initial_test_position_matrix)
final_alpha, final_length, final_center_position = find_wire_parameters(final_test_position_matrix)

center_to_center_position_matrix = create_position_matrix(initial_center_position[0], initial_center_position[1], final_center_position[0], final_center_position[1])
center_to_center_alpha, distance_to_travel, center_to_center_center_position = find_wire_parameters(center_to_center_position_matrix)

relative_angle = center_to_center_alpha - np.pi/2 - initial_alpha

position_matrix = coordinate_conversion(np.array([initial_test_position_matrix]) - initial_center_position, relative_angle, initial_center_position[0], initial_center_position[1])[0]

fig, ax = plt.subplots(figsize=(10, 10))

image_size_x = 2 # Update to scan size
image_size_y = 2 # Update to scan size

ax.set_xlim(0, image_size_x)  
ax.set_ylim(image_size_y, 0)
ax.set_aspect('equal', adjustable='box')
ax.grid(True)
ax.set_title('Initial and Transformed Vectors')
ax.set_xlabel('x')
ax.set_ylabel('y')

delta_x = 0.2
delta_y = 0.1
delta_theta = 0.1
wire_radius = 0.02
probe_radius = 0.012 
recession_dist = 0.1 

manipulation_matrix = move_nanowire(initial_test_position_matrix, final_center_position, final_alpha, delta_x, delta_y, delta_theta, wire_radius, probe_radius, recession_dist, ax=ax)

for i in range(len(manipulation_matrix)):
    ax = plot_PPVs(manipulation_matrix[i], ax=ax)

ax.get_children()[-11].set_label("Pushing vectors")

plot_vector(initial_test_position_matrix, ax=ax, label="Initial vector")
plot_vector(final_test_position_matrix, ax=ax, color='red', label="Final vector")
plot_vector(position_matrix, ax=ax, color='green', label="Intermediate vectors")
plot_vector(center_to_center_position_matrix, ax=ax, color='orange', label="Center to center vector")

ax.legend()

plt.savefig('Move.svg')  

with open('output.txt', 'w') as f:
    for operation_number in range(len(manipulation_matrix)):
        for polyline in manipulation_matrix[operation_number]:
            f.write('Polyline:\n')
            f.write(','.join(['[{:.8f}, {:.8f}]'.format(polyline[i, 0], polyline[i, 1]) for i in range(2)]))
            f.write('\n\n')

In [ ]:
##############
# User Input #
##############

initial_position_matrix = ask_for_coordinates("--- Initial position ---")
initial_alpha, initial_length, initial_center_position = find_wire_parameters(initial_position_matrix)

while True:
    final_alpha = input("Enter final angle [°] with respect to x axis: ")
    try:
        final_alpha = float(final_alpha)
        final_alpha = np.radians(final_alpha)
        break
    except ValueError:
        print("Please enter a valid float value for final angle.")

final_center_position = None
while final_center_position is None:
    final_center_position_input = input("Enter final center position: x, y")
    final_center_position_parts = final_center_position_input.split(',')
    try:
        final_center_position = np.array([float(final_center_position_parts[0]), float(final_center_position_parts[1])])
    except (ValueError, IndexError):
        print("Please enter two valid float values for final center position, separated by commas.")
        final_center_position = None

In [ ]:
############################
# Write for complete move #
############################

center_to_center_position_matrix = create_position_matrix(initial_center_position[0], initial_center_position[1], final_center_position[0], final_center_position[1])
center_to_center_alpha, distance_to_travel, center_to_center_center_position = find_wire_parameters(center_to_center_position_matrix)

relative_angle = center_to_center_alpha - np.pi/2 - initial_alpha

rotated_position_matrix = coordinate_conversion(np.array([initial_position_matrix]) - initial_center_position, relative_angle, initial_center_position[0], initial_center_position[1])[0]

fig, ax = plt.subplots(figsize=(30, 10))
image_size_x = 7 # Update to scan size
image_size_y = 7 # Update to scan size

ax.set_xlim(0, image_size_x)
ax.set_ylim(image_size_y, 0)  
ax.set_aspect('equal', adjustable='box')
ax.grid(True)
ax.set_title('Initial and Transformed Vectors')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Parameters to update
delta_x = 0.1
delta_y = 0.2 # Effective length of a pushing vector in the local coordinates of the nanowire.
delta_theta = 0.3
wire_radius = 0.02
probe_radius = 0.012 
recession_dist = 0.1 

manipulation_matrix = move_nanowire(initial_position_matrix, final_center_position, final_alpha, delta_x, delta_y, delta_theta, wire_radius, probe_radius, recession_dist, ax=ax)

for i in range(len(manipulation_matrix)):
    ax = plot_PPVs(manipulation_matrix[i], ax=ax)

plot_vector(initial_position_matrix, ax=ax)
plot_vector(rotated_position_matrix, ax=ax, color='green')
plot_vector(center_to_center_position_matrix, ax=ax, color='orange')

with open('move.txt', 'w') as f:
    for operation_number in range(len(manipulation_matrix)):
        for polyline in manipulation_matrix[operation_number]:
            f.write('Polyline:\n')
            f.write(','.join('{:.8f}, {:.8f}'.format(polyline[i, 0], polyline[i, 1]) for i in range(2)))
            f.write('\n\n')

In [ ]:
###########################
# Write for rotation only #
###########################

final_alpha = 0 # Change to the desired angle here. Taken with respect to the x axis and clockwise.
final_alpha = np.radians(final_alpha)

relative_angle = final_alpha - initial_alpha

rotated_position_matrix = coordinate_conversion(np.array([initial_position_matrix]) - initial_center_position, relative_angle, initial_center_position[0], initial_center_position[1])[0]

fig, ax = plt.subplots(figsize=(30, 10))
image_size_x = 4 # Update to scan size
image_size_y = 4 # Update to scan size

ax.set_xlim(0, image_size_x)  
ax.set_ylim(image_size_y, 0)  
ax.set_aspect('equal', adjustable='box')
ax.grid(True)
ax.set_title('Initial and Transformed Vectors')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Parameters to update
delta_x = 0.1
delta_theta = 0.3
wire_radius = 0.02 
probe_radius = 0.012 
recession_dist = 0.1 

manipulation_matrix = rotate_nanowire(initial_position_matrix, final_alpha, delta_x, delta_theta, wire_radius, probe_radius, recession_dist, do_plot=True, ax=ax)

for i in range(len(manipulation_matrix)):
    ax = plot_PPVs(manipulation_matrix[i], ax=ax)

plot_vector(initial_position_matrix, ax=ax)
plot_vector(rotated_position_matrix, ax=ax, color='red')

with open('rotation.txt', 'w') as f:
    for operation_number in range(len(manipulation_matrix)):
        for polyline in manipulation_matrix[operation_number]:
            f.write('Polyline:\n')
            f.write(','.join('{:.8f}, {:.8f}'.format(polyline[i, 0], polyline[i, 1]) for i in range(2)))
            f.write('\n\n')

In [ ]:
##############################
# Write for translation only #
##############################

distance_to_travel = 2
direction_y = -1

translation_alpha = np.pi/2 * direction_y  + initial_alpha
displacement_vector = np.array([np.cos(translation_alpha), np.sin(translation_alpha)]) * distance_to_travel

final_center_position = initial_center_position + displacement_vector

translated_position_matrix = coordinate_conversion(np.array([initial_position_matrix]), 0, final_center_position[0] - initial_center_position[0], final_center_position[1] - initial_center_position[1])[0]

fig, ax = plt.subplots(figsize=(30, 10))
image_size_x = 7 # Update to scan size
image_size_y = 7 # Update to scan size

ax.set_xlim(0, image_size_x)  
ax.set_ylim(image_size_y, 0)  
ax.set_aspect('equal', adjustable='box')
ax.grid(True)
ax.set_title('Initial and Transformed Vectors')
ax.set_xlabel('x')
ax.set_ylabel('y')

# Parameters to update
delta_x = 0.1
delta_y = 0.2 # Effective length of a pushing vector in the local coordinates of the nanowire.
wire_radius = 0.02 
probe_radius = 0.012 
recession_dist = 0.1 

manipulation_matrix = translate_nanowire(initial_position_matrix, final_center_position, delta_x, delta_y, wire_radius, probe_radius, recession_dist)

for i in range(len(manipulation_matrix)):
    ax = plot_PPVs(manipulation_matrix[i], ax=ax)

plot_vector(initial_position_matrix, ax=ax)
plot_vector(translated_position_matrix, ax=ax, color='red')

with open('translation.txt', 'w') as f:
    for operation_number in range(len(manipulation_matrix)):
        for polyline in manipulation_matrix[operation_number]:
            f.write('Polyline:\n')
            f.write(','.join('{:.8f}, {:.8f}'.format(polyline[i, 0], polyline[i, 1]) for i in range(2)))
            f.write('\n\n')